In [0]:
def create_model():

    FCN_CLASSES = 21

    #(samples, channels, rows, cols)
    input_img = Input(shape=(3, 224, 224))
    #(3*224*224)
    x = Convolution2D(64, 3, 3, activation='relu',border_mode='same')(input_img)
    x = Convolution2D(64, 3, 3, activation='relu',border_mode='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #(64*112*112)
    x = Convolution2D(128, 3, 3, activation='relu',border_mode='same')(x)
    x = Convolution2D(128, 3, 3, activation='relu',border_mode='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #(128*56*56)
    x = Convolution2D(256, 3, 3, activation='relu',border_mode='same')(x)
    x = Convolution2D(256, 3, 3, activation='relu',border_mode='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #(256*56*56)

    #split layer
    p3 = x
    p3 = Convolution2D(FCN_CLASSES, 1, 1,activation='relu')(p3)
    #(21*28*28)

    x = Convolution2D(512, 3, 3, activation='relu',border_mode='same')(x)
    x = Convolution2D(512, 3, 3, activation='relu',border_mode='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #(512*14*14)

    #split layer
    p4 = x
    p4 = Convolution2D(FCN_CLASSES, 1, 1, activation='relu')(p4)
    p4 = Deconvolution2D(FCN_CLASSES, 4, 4,
            output_shape=(None, FCN_CLASSES, 30, 30),
            subsample=(2, 2),
            border_mode='valid')(p4)
    p4 = Cropping2D(cropping=((1, 1), (1, 1)))(p4)

    #(21*28*28)

    x = Convolution2D(512, 3, 3, activation='relu',border_mode='same')(x)
    x = Convolution2D(512, 3, 3, activation='relu',border_mode='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #(512*7*7)

    p5 = x
    p5 = Convolution2D(FCN_CLASSES, 1, 1, activation='relu')(p5)
    p5 = Deconvolution2D(FCN_CLASSES, 8, 8,
            output_shape=(None, FCN_CLASSES, 32, 32),
            subsample=(4, 4),
            border_mode='valid')(p5)
    p5 = Cropping2D(cropping=((2, 2), (2, 2)))(p5)
    #(21*28*28)

    # merge scores
    merged = merge([p3, p4, p5], mode='sum')
    x = Deconvolution2D(FCN_CLASSES, 16, 16,
            output_shape=(None, FCN_CLASSES, 232, 232),
            subsample=(8, 8),
            border_mode='valid')(merged)
    x = Cropping2D(cropping=((4, 4), (4, 4)))(x)
    x = Reshape((FCN_CLASSES,224*224))(x)
    x = Permute((2,1))(x)
    out = Activation("softmax")(x)
    #(21,224,224)
    model = Model(input_img, out)
    return model

In [0]:
def binarylab(labels):
    x = np.zeros([224,224,21])
    for i in range(224):
        for j in range(224):
            x[i,j,labels[i][j]]=1
    return x

In [0]:
def generate_arrays_from_file(path):
    while 1:
        with open(path,"r") as f:
            names = f.readlines()
        ls = [line.rstrip('\n') for line in names]
        for name in ls:
            img, target = process_line(name)
            yield (img, target)

In [0]:
def train(path):
    model = create_model()
    model.compile(loss="categorical_crossentropy",
                  optimizer='adadelta',
                  metrics=["accuracy"])
    model.fit_generator(generate_arrays_from_file(path),
        samples_per_epoch=1450, nb_epoch=100)
    model.save_weights('weights.hdf5')
    return